# Machado de Assis Bot
**Portuguese Text Generation with Artificial Neural Networks**
<img src = "MachadoAssisBot.png">

- Project developed by Marcelo Rovai, based on lessons learned with the great teacher [José Portilla](https://www.udemy.com/course/complete-tensorflow-2-and-keras-deep-learning-bootcamp/#instructor-1) during his fantastic course: [Complete Tensorflow 2 and Keras Deep Learning Bootcamp](https://www.udemy.com/course/complete-tensorflow-2-and-keras-deep-learning-bootcamp/#instructor-1)
- The Artificial network developed on this project, generates text, character by character. Please see this following site, for a great general visual explanation: [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and also check the [TF site for details](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/text_generation.ipynb). 


## Import Libraries

In [ ]:
# Should be run when GOOGLE COLLAB is used
%tensorflow_version 2.x

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Embedding,Dropout,GRU
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.models import load_model

In [2]:
tf.__version__

'2.2.0'

## Step 1: The Data

Machado de Assis original XIX century books can be downloaded from: [Gutenberg Project](https://www.gutenberg.org/)

### Verifying books

In [3]:
ls machado/

casmurro.txt    mao_luva.txt    memorias.txt    quincas.txt
esau_jacob.txt  memorial.txt    papeis.txt


In [29]:
path = './machado/'

In [101]:
text1 = open(path+'casmurro.txt', 'r').read()
text2 = open(path+'mao_luva.txt', 'r').read()
text3 = open(path+'memorias.txt', 'r').read()
text4 = open(path+'quincas.txt', 'r').read()
text5 = open(path+'esau_jacob.txt', 'r').read()
text6 = open(path+'memorial.txt', 'r').read()
text7 = open(path+'papeis.txt', 'r').read()
text = text1+text2+text3+text4+text5+text6+text7
len(text)

2418451

In [103]:
print(text[:500])

DOM CASMURRO

I

Do titulo.

Uma noite destas, vindo da cidade para o Engenho Novo, encontrei no
trem da Central um rapaz aqui do bairro, que eu conheço de vista e
de chapéo. Comprimentou-me, sentou-se ao pé de mim, falou da lua e
dos ministros, e acabou recitando-me versos. A viagem era curta, e os
versos póde ser que não fossem inteiramente maus. Succedeu, porém, que
como eu estava cançado, fechei os olhos tres ou quatro vezes; tanto
bastou para que elle interrompesse a leitura e mettesse os v


In [47]:
print(text[-500:])

s principaes medicos
da côrte; foi necessario recorrer á simulação, e dal-os, emfim, como
receitados por um ignorantão do tempo. Mas era tarde. A morte levou-o,
ao cabo de duas semanas.

--Joaquim Soares? bradou attonito o cunhado, ao saber da verba
testamentaria do defunto, ordenando que o caixão fosse fabricado por
aquelle industrial. Mas os caixões d'esse sujeito não prestam para
nada, e...

--Paciencia! interrompeu a mulher; a vontade do mano ha de comprir-se.


FIM DA VERBA TESTAMENTARIA





### Generating and saving a single file (corpus)

In [108]:
machado = open('machado.txt','w')

In [109]:
machado.write(text)

2418451

In [110]:
machado.close()

### Understanding unique characters

In [113]:
text = open('machado.txt', 'r').read()
len(text)

2418451

In [50]:
# The unique characters in the file
vocab = sorted(set(text))
print(vocab)
len(vocab)

['\n', ' ', '!', '"', '$', '&', "'", '(', ')', '*', '+', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '§', '«', '°', '»', 'À', 'Á', 'Ã', 'Ç', 'É', 'Ê', 'Í', 'Ó', 'Ú', 'à', 'á', 'â', 'ã', 'æ', 'ç', 'è', 'é', 'ê', 'í', 'î', 'ñ', 'ò', 'ó', 'ô', 'õ', 'ú', 'û', 'œ', '—', '’']


117

## Step 2: Text Processing

### Text Vectorization

We know a neural network can't take in the raw string data, we need to assign numbers to each character. Let's create two dictionaries that can go from numeric index to character and character to numeric index.

In [51]:
char_to_ind = {char:ind for ind, char in enumerate(vocab)}

In [52]:
char_to_ind

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '$': 4,
 '&': 5,
 "'": 6,
 '(': 7,
 ')': 8,
 '*': 9,
 '+': 10,
 ',': 11,
 '-': 12,
 '.': 13,
 '0': 14,
 '1': 15,
 '2': 16,
 '3': 17,
 '4': 18,
 '5': 19,
 '6': 20,
 '7': 21,
 '8': 22,
 '9': 23,
 ':': 24,
 ';': 25,
 '=': 26,
 '?': 27,
 'A': 28,
 'B': 29,
 'C': 30,
 'D': 31,
 'E': 32,
 'F': 33,
 'G': 34,
 'H': 35,
 'I': 36,
 'J': 37,
 'K': 38,
 'L': 39,
 'M': 40,
 'N': 41,
 'O': 42,
 'P': 43,
 'Q': 44,
 'R': 45,
 'S': 46,
 'T': 47,
 'U': 48,
 'V': 49,
 'W': 50,
 'X': 51,
 'Y': 52,
 'Z': 53,
 '[': 54,
 ']': 55,
 '_': 56,
 'a': 57,
 'b': 58,
 'c': 59,
 'd': 60,
 'e': 61,
 'f': 62,
 'g': 63,
 'h': 64,
 'i': 65,
 'j': 66,
 'k': 67,
 'l': 68,
 'm': 69,
 'n': 70,
 'o': 71,
 'p': 72,
 'q': 73,
 'r': 74,
 's': 75,
 't': 76,
 'u': 77,
 'v': 78,
 'w': 79,
 'x': 80,
 'y': 81,
 'z': 82,
 '§': 83,
 '«': 84,
 '°': 85,
 '»': 86,
 'À': 87,
 'Á': 88,
 'Ã': 89,
 'Ç': 90,
 'É': 91,
 'Ê': 92,
 'Í': 93,
 'Ó': 94,
 'Ú': 95,
 'à': 96,
 'á': 97,
 'â': 98,
 'ã': 99,
 'æ': 100

In [53]:
char_to_ind['H']

35

In [54]:
ind_to_char = np.array(vocab)

In [55]:
ind_to_char

array(['\n', ' ', '!', '"', '$', '&', "'", '(', ')', '*', '+', ',', '-',
       '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';',
       '=', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K',
       'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X',
       'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h',
       'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
       'v', 'w', 'x', 'y', 'z', '§', '«', '°', '»', 'À', 'Á', 'Ã', 'Ç',
       'É', 'Ê', 'Í', 'Ó', 'Ú', 'à', 'á', 'â', 'ã', 'æ', 'ç', 'è', 'é',
       'ê', 'í', 'î', 'ñ', 'ò', 'ó', 'ô', 'õ', 'ú', 'û', 'œ', '—', '’'],
      dtype='<U1')

In [56]:
ind_to_char[35]

'H'

In [57]:
encoded_text = np.array([char_to_ind[c] for c in text])

In [58]:
encoded_text

array([31, 42, 40, ...,  0,  0,  0])

In [59]:
encoded_text.shape

(2418451,)

We now have a mapping we can use to go back and forth from characters to numerics.

In [60]:
sample = text[:500]
sample

'DOM CASMURRO\n\nI\n\nDo titulo.\n\nUma noite destas, vindo da cidade para o Engenho Novo, encontrei no\ntrem da Central um rapaz aqui do bairro, que eu conheço de vista e\nde chapéo. Comprimentou-me, sentou-se ao pé de mim, falou da lua e\ndos ministros, e acabou recitando-me versos. A viagem era curta, e os\nversos póde ser que não fossem inteiramente maus. Succedeu, porém, que\ncomo eu estava cançado, fechei os olhos tres ou quatro vezes; tanto\nbastou para que elle interrompesse a leitura e mettesse os v'

In [61]:
encoded_text[:500]

array([ 31,  42,  40,   1,  30,  28,  46,  40,  48,  45,  45,  42,   0,
         0,  36,   0,   0,  31,  71,   1,  76,  65,  76,  77,  68,  71,
        13,   0,   0,  48,  69,  57,   1,  70,  71,  65,  76,  61,   1,
        60,  61,  75,  76,  57,  75,  11,   1,  78,  65,  70,  60,  71,
         1,  60,  57,   1,  59,  65,  60,  57,  60,  61,   1,  72,  57,
        74,  57,   1,  71,   1,  32,  70,  63,  61,  70,  64,  71,   1,
        41,  71,  78,  71,  11,   1,  61,  70,  59,  71,  70,  76,  74,
        61,  65,   1,  70,  71,   0,  76,  74,  61,  69,   1,  60,  57,
         1,  30,  61,  70,  76,  74,  57,  68,   1,  77,  69,   1,  74,
        57,  72,  57,  82,   1,  57,  73,  77,  65,   1,  60,  71,   1,
        58,  57,  65,  74,  74,  71,  11,   1,  73,  77,  61,   1,  61,
        77,   1,  59,  71,  70,  64,  61, 101,  71,   1,  60,  61,   1,
        78,  65,  75,  76,  57,   1,  61,   0,  60,  61,   1,  59,  64,
        57,  72, 103,  71,  13,   1,  30,  71,  69,  72,  74,  6

## Step 3: Creating Batches

Overall what we are trying to achieve is to have the model predict the next highest probability character given a historical sequence of characters. Its up to us (the user) to choose how long that historic sequence. Too short a sequence and we don't have enough information (e.g. given the letter "a" , what is the next character) , too long a sequence and training will take too long and most likely overfit to sequence characters that are irrelevant to characters farther out. While there is no correct sequence length choice, you should consider the text itself, how long normal phrases are in it, and a reasonable idea of what characters/words are relevant to each other.

In [62]:
print(text[:500])

DOM CASMURRO

I

Do titulo.

Uma noite destas, vindo da cidade para o Engenho Novo, encontrei no
trem da Central um rapaz aqui do bairro, que eu conheço de vista e
de chapéo. Comprimentou-me, sentou-se ao pé de mim, falou da lua e
dos ministros, e acabou recitando-me versos. A viagem era curta, e os
versos póde ser que não fossem inteiramente maus. Succedeu, porém, que
como eu estava cançado, fechei os olhos tres ou quatro vezes; tanto
bastou para que elle interrompesse a leitura e mettesse os v


In [63]:
line = "Uma noite destas, vindo da cidade para o Engenho Novo, encontrei no"

In [64]:
len(line)

67

In [66]:
par = """Uma noite destas, vindo da cidade para o Engenho Novo, encontrei no
trem da Central um rapaz aqui do bairro, que eu conheço de vista e
de chapéo."""

In [67]:
len(par)

145

We will use 120 as a general paragraph

### Training Sequences

The actual text data will be the text sequence shifted one character forward. For example:

Sequence In: "Hello my nam"
Sequence Out: "ello my name"


We can use the `tf.data.Dataset.from_tensor_slices` function to convert a text vector into a stream of character indices.

In [68]:
seq_len = 120

In [69]:
total_num_seq = len(text)//(seq_len+1)

In [70]:
total_num_seq

19987

In [71]:
# Create Training Sequences
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

In [72]:
type(char_dataset)

tensorflow.python.data.ops.dataset_ops.TensorSliceDataset

In [74]:
# for i in char_dataset.take(500):
#      print(ind_to_char[i.numpy()])

The **batch** method converts these individual character calls into sequences we can feed in as a batch. We use seq_len+1 because of zero indexing. Here is what drop_remainder means:

drop_remainder: (Optional.) A `tf.bool` scalar `tf.Tensor`, representing
    whether the last batch should be dropped in the case it has fewer than
    `batch_size` elements; the default behavior is not to drop the smaller
    batch.


In [75]:
sequences = char_dataset.batch(seq_len+1, drop_remainder=True)

In [76]:
sequences

<BatchDataset shapes: (121,), types: tf.int64>

Now that we have our sequences, we will perform the following steps for each one to create our target text sequences:

1. Grab the input text sequence
2. Assign the target text sequence as the input text sequence shifted by one step forward
3. Group them together as a tuple

In [77]:
def create_seq_targets(seq): # Hello my name
    input_txt = seq[:-1]     # Hello my nam
    target_txt = seq[1:]     # ello my name
    return input_txt, target_txt

In [78]:
dataset = sequences.map(create_seq_targets)

In [42]:
# for test only

In [79]:
for input_txt, target_txt in  dataset.take(2):
    print(input_txt.numpy())
    print(''.join(ind_to_char[input_txt.numpy()]))
    print('\n')
    print(target_txt.numpy())
    # There is an extra whitespace!
    print(''.join(ind_to_char[target_txt.numpy()]))

[31 42 40  1 30 28 46 40 48 45 45 42  0  0 36  0  0 31 71  1 76 65 76 77
 68 71 13  0  0 48 69 57  1 70 71 65 76 61  1 60 61 75 76 57 75 11  1 78
 65 70 60 71  1 60 57  1 59 65 60 57 60 61  1 72 57 74 57  1 71  1 32 70
 63 61 70 64 71  1 41 71 78 71 11  1 61 70 59 71 70 76 74 61 65  1 70 71
  0 76 74 61 69  1 60 57  1 30 61 70 76 74 57 68  1 77 69  1 74 57 72 57]
DOM CASMURRO

I

Do titulo.

Uma noite destas, vindo da cidade para o Engenho Novo, encontrei no
trem da Central um rapa


[42 40  1 30 28 46 40 48 45 45 42  0  0 36  0  0 31 71  1 76 65 76 77 68
 71 13  0  0 48 69 57  1 70 71 65 76 61  1 60 61 75 76 57 75 11  1 78 65
 70 60 71  1 60 57  1 59 65 60 57 60 61  1 72 57 74 57  1 71  1 32 70 63
 61 70 64 71  1 41 71 78 71 11  1 61 70 59 71 70 76 74 61 65  1 70 71  0
 76 74 61 69  1 60 57  1 30 61 70 76 74 57 68  1 77 69  1 74 57 72 57 82]
OM CASMURRO

I

Do titulo.

Uma noite destas, vindo da cidade para o Engenho Novo, encontrei no
trem da Central um rapaz
[  1  57  73  77  65   1

### Generating training batches

Now that we have the actual sequences, we will create the batches, we want to shuffle these sequences into a random order, so the model doesn't overfit to any section of the text, but can instead generate characters given any seed text.

In [80]:
# Batch size
batch_size = 128

# Buffer size to shuffle the dataset so it doesn't attempt to shuffle
# the entire sequence in memory. Instead, it maintains a buffer in which it shuffles elements
buffer_size = 10000

dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

In [81]:
dataset

<BatchDataset shapes: ((128, 120), (128, 120)), types: (tf.int64, tf.int64)>

## Step 4: Creating the Model

We will use an LSTM based model with a few extra features, including an embedding layer to start off with and **two** LSTM layers. We based this model architecture off the [DeepMoji](https://deepmoji.mit.edu/) and the original source code can be found [here](https://github.com/bfelbo/DeepMoji).

The embedding layer will serve as the input layer, which essentially creates a lookup table that maps the numbers indices of each character to a vector with "embedding dim" number of dimensions. As you can imagine, the larger this embedding size, the more complex the training. This is similar to the idea behind word2vec, where words are mapped to some n-dimensional space. Embedding before feeding straight into the LSTM usually leads to more realisitic results.

In [82]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embed_dim = 64

# Number of RNN units
rnn_neurons = 1026

Now let's create a function that easily adapts to different variables as shown above.

### Setting up Loss Function

For our loss we will use sparse categorical crossentropy, which we can import from Keras. We will also set this as logits=True

https://datascience.stackexchange.com/questions/41921/sparse-categorical-crossentropy-vs-categorical-crossentropy-keras-accuracy

In [86]:
def sparse_cat_loss(y_true,y_pred):
    return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

In [87]:
def create_model(vocab_size, embed_dim, rnn_neurons, batch_size):
    model = Sequential()
    model.add(Embedding(vocab_size, embed_dim,batch_input_shape=[batch_size, None]))
    model.add(GRU(rnn_neurons,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'))
    # Final Dense Layer to Predict
    model.add(Dense(vocab_size))
    model.compile(optimizer='adam', loss=sparse_cat_loss) 
    return model

In [88]:
model = create_model(
  vocab_size = vocab_size,
  embed_dim=embed_dim,
  rnn_neurons=rnn_neurons,
  batch_size=batch_size)

In [89]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, None, 64)           7488      
_________________________________________________________________
gru (GRU)                    (128, None, 1026)         3361176   
_________________________________________________________________
dense (Dense)                (128, None, 117)          120159    
Total params: 3,488,823
Trainable params: 3,488,823
Non-trainable params: 0
_________________________________________________________________


## Step 5: Training the model

Let's make sure everything is ok with our model before we spend too much time training! Let's pass in a batch to confirm the model currently predicts random characters without any training.



In [90]:
for input_example_batch, target_example_batch in dataset.take(1):

  # Predict off some random batch
  example_batch_predictions = model(input_example_batch)

  # Display the dimensions of the predictions
  print(example_batch_predictions.shape, " <=== (batch_size, sequence_length, vocab_size)")


(128, 120, 117)  <=== (batch_size, sequence_length, vocab_size)


In [54]:
example_batch_predictions[0]

<tf.Tensor: shape=(120, 84), dtype=float32, numpy=
array([[ 1.8190127e-04, -1.6419976e-03,  3.9721478e-04, ...,
        -9.5687859e-04,  3.0489508e-03,  8.8712717e-05],
       [ 1.1579794e-02, -5.5200760e-03,  7.0615485e-03, ...,
         9.6486701e-04,  1.5863801e-03,  6.0194358e-04],
       [ 4.2760507e-03, -1.9974667e-03,  8.0016572e-03, ...,
         2.1912388e-03, -2.1127991e-03,  3.4331047e-04],
       ...,
       [ 7.5974653e-04, -5.6152726e-03,  5.9420167e-04, ...,
         7.5961314e-03, -5.2516577e-03, -5.8296616e-03],
       [ 1.4868858e-03, -3.1611430e-03, -4.7921604e-03, ...,
         7.3029827e-03, -4.9130362e-03, -5.5329129e-04],
       [ 8.2738942e-04, -3.1001735e-03, -1.8603078e-03, ...,
         2.8914216e-03,  3.6097656e-04, -6.5167190e-04]], dtype=float32)>

In [93]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)

In [94]:
#sampled_indices

In [95]:
# Reformat to not be a lists of lists
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [96]:
sampled_indices

array([ 88,   4,   1,  57, 104,  49,  36,  90, 111,  87,  89,  33,  74,
         7,  39, 109,  82,   0,  70,  87,  22, 112,  26,  18,  93,   6,
        79,   1,  41,  60,  51,  55,  12,  45,  10,  45,  50,  90,  75,
        63,   8,  18,  56,  84,  70,  61,  36, 109, 106,  45,  24,  54,
        52,  67,  61,  35,  24,  13, 100,  91, 104,  46,  74,  75,  39,
        77,  49,  83,  34,  39,  10,  32,  88,  79, 105,  37,  41,  40,
        93,  52,  99,  94,  37,  23,  75,  91,  23, 100,  48,  32,  53,
        89,  18,  24, 113,  89,  46,  39,  27,  45,  21,  93,  42,  43,
        14,  61,  60,  65, 114,  95,  37,  91,  84,  14,  54,  57,  77,
         8,  63,  64])

In [97]:
print("Given the input seq: \n")
print("".join(ind_to_char[input_example_batch[0]]))
print('\n')
print("Next Char Predictions: \n")
print("".join(ind_to_char[sampled_indices ]))


Given the input seq: 

, temperando o mal com
a opinião anti-russa, dava á podridão das suas carnes um reflexo
espiritual que as consolava. Ha 


Next Char Predictions: 

Á$ aêVIÇõÀÃFr(Lóz
nÀ8ú=4Í'w NdX]-R+RWÇsg)4_«neIóîR:[YkeH:.æÉêSrsLuV§GL+EÁwíJNMÍYãÓJ9sÉ9æUEZÃ4:ûÃSL?R7ÍOP0ediœÚJÉ«0[au)gh


After confirming the dimensions are working, let's train our network!

In [98]:
epochs = 30

**Model should be trained using GPU on Google Collabs. It is faster than a CPU**

In [ ]:
# model.fit(dataset,epochs=epochs)

In [ ]:
# model.save('machado_gen_mjr.h5') Model Trained on Google Colabs

## Step 6: Generating text

Currently our model only expects 128 sequences at a time. We can create a new model that only expects a batch_size=1. We can create a new model with this batch size, then load our saved models weights. Then call .build() on the model:

In [115]:
model = create_model(vocab_size, embed_dim, rnn_neurons, batch_size=1)
model.load_weights('machado_gen_mjr.h5')
model.build(tf.TensorShape([1, None]))

In [116]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 64)             7488      
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1026)           3361176   
_________________________________________________________________
dense_1 (Dense)              (1, None, 117)            120159    
Total params: 3,488,823
Trainable params: 3,488,823
Non-trainable params: 0
_________________________________________________________________


In [117]:
def generate_text(model, start_seed,gen_size=100,temp=1.0):
    '''
    model: Trained Model to Generate Text
    start_seed: Intial Seed text in string form
    gen_size: Number of characters to generate

    Basic idea behind this function is to take in some seed text, format it so
    that it is in the correct shape for our network, then loop the sequence as
    we keep adding our own predicted characters. Similar to our work in the RNN
    time series problems.
    '''

    # Number of characters to generate
    num_generate = gen_size

    # Vectorizing starting seed text
    input_eval = [char_to_ind[s] for s in start_seed]

    # Expand to match batch format shape
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty list to hold resulting generated text
    text_generated = []

    # Temperature effects randomness in our resulting text
    # The term is derived from entropy/thermodynamics.
    # The temperature is used to effect probability of next characters.
    # Higher probability == lesss surprising/ more expected
    # Lower temperature == more surprising / less expected

    temperature = temp

    # Here batch size == 1
    model.reset_states()

    for i in range(num_generate):
        # Generate Predictions
        predictions = model(input_eval)

        # Remove the batch shape dimension
        predictions = tf.squeeze(predictions, 0)

        # Use a cateogircal disitribution to select the next character
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pass the predicted charracter for the next input
        input_eval = tf.expand_dims([predicted_id], 0)

        # Transform back to character letter
        text_generated.append(ind_to_char[predicted_id])

    return (start_seed + ''.join(text_generated))

In [123]:
print(generate_text(model,"Maria",gen_size=1000))

Maria»0:

«A moça foi bem, porque fica um tanto assim que,
pendelo. Baptista Sophia onde vimos dahi a todos.

--Os cabellos não vieram respirou constrangida no jardim de Quincha,
fitavam a ir adeante de si, apostou-se, passea á revoctadão. Não podia
resistrar mal e generopha. A lua, só então olhavam caminhal a differença; eram buscadas, ao vel-a noite de Humanitas. Os outros homens
que trocassemos um defeito mesmo mal. Naturalmente, não se explicam os defeitos na tenra. Meu pae
fará o lenço, sem nada, nem os olhos, e passeava a tempo. Tendo-lhe descobrir. Mas nem só
os passessos, raros nunca!

       *       *       *       *       *

1 de Novem possumavam o interesse.

Jantavam as costas do vostura, e o furor, a sou o
iam, falando dos seus olhos. Um bom chegou ao pescoço, onde os cinco
contos, não sei se tinha ir para si. Então os encontrou _semblem de Maio.

Reverante, olhava para a Italia e da deloridade
ás orelhas de trinta. Era uma sombra de uma secula menter, deitar-lhe o tempo d

In [126]:
print(generate_text(model," CASMURRO ",gen_size=1000))

 CASMURRO III


Pedro e Paulo viu-o; delibentei, que
bem passam a aurora, provavelmente o que foi por
isso. Não me lembra, bem a senhora tinha ao proprio Rubião que vae sair do
ceitaço. Demaia,
consolei-o a dama, que é ella que ra viste que fariam vencedor.
Mal podia continuar. Bem o carinho acclamada estava abençoa não se
podem nunca mais. Ouviu-aspiar ao gospo, de natureza, consultar o
tratem, e perde--2       *       *       *  Deus a exageração do
cosinheiro, obrigado o beiço, trocam para a filha, saiu
obediente, ou o que quer que festejava, par de nhão, mas a lante marcava a aborrecer-se,
agradaveis que
isto presente, o moço secreto dos inglezes, o presente, um verdadeiro e proximo,
quando lhe leu uma sensação peccada de Lobo Neves que queira,
e effectivamente ajuda-me a quatro horas,
desde o desejo de ir baixado, este é o frequentemente de figurar os que
batiam á minha humidina e bom, como um tal caso prazer com prosa.
Justamente o seu criado, por sedo o resto. Casa Braz Cubas_, 

In [127]:
print(generate_text(model," Rio de Janeiro  ",gen_size=5000))

 Rio de Janeiro     tudo, e é uma doce recebida no seculo. Mas o
alienista me deu dous annos. Já não trazia talvez de manhã. Como
veiu, acharia em minha casa, de péuco tempe; conspira algumas semanas todas, um
fino, e fazia pagar no meio do charuto, mas do laço de Lisboa e de Japhet e Senhor que lhe fiz
um bello te podia attural-o.

Um preceder não ser namorar, uma vez presa, a clausuza, que os jo
agora constante, Flora prometteu-se aos seus ouvidos, as respostas
largas, é certo, e eu prometti não denas saudades penos que D.
Benedicta não tem o senhor era bonita. Eu mesmo não faça de nós; temo
rasgado ao edio.

Eram onzes. Olhavam e cabeças, e prologos de Menriso. O Humanitismo homem se não deu ficar
e teimam nada, e, ainda cá porque...

--Oh! Carmo, é objecção com Guiomar, disse Venuzindo ao essenciado
esse curioso. Um antigo bom, creio até á experielha
de a vestidos á não faltar-me no céu. E contei-lhe o rosto de
mofeis, até que só instifava as arvores nem de Escobar, mas a Capitú, i

In [128]:
print(generate_text(model,"A LUVA DE CASMURRO ",gen_size=5000))

A LUVA DE CASMURRO IM não foi sentar-se n'um collete,
tão cedo estivesse á dubição, sem deitar a noite visava-lhe a
ideia da mulher, ao vel-a preso um relevo dae passou. Só depois de a pedir lhe
não fica que as mimosadios de Camõista.

Vencer deste, quaesquer que fosse, mas apenas dizia
quatro horas da tarde... Bonra, diante nos seus barbeiros. Quem não foi nada
condemnariam, e, se não foi uma atternação de algarismos, sem o soube, do
barbeiro, vá propria. Quando tem muito zemando,
respondeu que este fugir lhe bons expos os meios de
secretaria. Mas devo lhe
digo uma. Passou a juro.

--Venha? perguntava-as com trincada, e uma administração fizeram
em quebro hoje ou a andar, porque não faz mal impossivel. Olhou alguns
instantes, mas vão profundamente, e diga que Deus, na vespera.




CAPITULO LIII


Naquella
noite amanheceu o motivo da patria, com
que respondente emquanto Não. Você vae Luiz Alves.

--Vá lá. Se é vista alguma cousa.

--Nem eu, nem em fantos, conclui por
medico-se ou vagab

In [129]:
print(generate_text(model,"A LUVA DE CASMURRO ",gen_size=5000))

A LUVA DE CASMURRO DO ANESADOS DA MADOVII


Adiava para rectar a exposição, a
impossibilidade de commoscaça aborrece a idéa francezamente
a filha, parava, lembrou-se, e seguiram. José Dias desculpe rapaz, e a propria
edição propriamente ouvi outro. Talvez poí nelle um formito,
uma filha de familia, suspirando a causa da minha cosida, depois
toda a ausencia de Pedro. Mas eu tenão acompanharia um turbo na
cidade e da especie, como era pilherna?

Rubião, grotante, designado, a regra particular, o cerebro de
alma que, por não saber a conversação foi cousa, mas
é mais alegre e fecho de que todos restamente meio dinheiro, se
puder almoçar, e vi que esta syprovanda já lá peço a da minha velha: «Anniscou taes do outro ebige,
não menos que o dia do casamento. O protector deres de
cabeça, respondeu-lhe que vielimento resurra a uma mesa. Não
lhe tivesse o farto de duas janellas, vendo o titulo possivel. D. Carmo não disse a colera fosse
critica do moço, ferindo, sentado na rua, começando ao golpe

In [131]:
print(generate_text(model,"A LUVA DE CASMURRO ",gen_size=5000, temp=0.5))

A LUVA DE CASMURRO II


A missa do _coupé_ e um presente e o
governo devia cazar logo no papel, a morte do autor, e todos os seus
considerados de alegria. Era um espirito de vinte e cinco annos, e eu não
estou alguns passos no cerebro, como de outra cousa. Deus me disse:

--Não digo que não. Se eu tivesse a intenção de um probosito. Palha acudiu a mulher, não
havia nada. A noite vinha tambem para o seminario, tinha o aspecto
do partido recto e de restaurar a minha mãe e
do pae, pela primeira vez, a menor destinada a
dispensar o chapéo, esperou que não vinhas com as suas mãos de creanças. A manhã della
chegasse a baroneza e a maneira desta divida. Parece que é casada.

--Está bom, perdoa-lhe de todos os lados, a vida de que o comprar para o meu quarto de hora,
e contavam com o fim de a anterior, e, a parede pouco tempo a alma de pessoas que definitivamente lhe
interessam a menos para mim. De quando em quando, esses dous annos de conversação para o fim de deixar nenhuma
pessoa que se dis